## Training Positive and Unlabeled Materials Machine Learning (PUMML) models on Materials Project data
This notebook shows how to:
* Access material structures from the Materials Project (MP) using the Materials API (MAPI) or figshare and label them as synthesized (1) or not-yet synthesized (0).  
* Featurize materials and pre-process data for machine learning.
* Train PUMML models to predict material synthesizability.
* Evaluate and interpret model outputs.

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

from monty.serialization import loadfn, dumpfn

from pymatgen.ext.matproj import MPRester
from pymatgen.core import Structure

### Accessing MP data

You can access all MP structures (as of 04-24-2020) directly from figshare: https://figshare.com/account/home#/collections/4952793.  

However, the MP is constantly being updated and new structures are added. It is highly recommended that you use the MAPI to pull structure data that you are interested in. 

This code shows how to apply some criteria (e.g., ignore compounds with f-block elements), get MP IDs (which does not take much time), and then download structures in chunks (time-consuming).

In [ ]:
# Treat materials with f-block electrons separately.
fblock = ['Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 
         'Tm', 'Yb', 'Lu', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk',
         'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr']

criteria = {"elements": {"$nin": fblock}}  # exclude fblock

In [ ]:
# https://wiki.materialsproject.org/The_Materials_API
mpids = []
with MPRester() as m:  # include api key as argument or configure with pmg command line 
    mp_ids = m.query(criteria, ["material_id"], chunk_size=0)

In [ ]:
# Tag with date collected
today = datetime.today().strftime('%Y-%m-%d')

mp_ids = [mpid['material_id'] for mpid in mp_ids]
dumpfn(mp_ids, "mp_ids_%s.json" % (today))

In [ ]:
mp_ids = loadfn('mp_ids_%s.json' %(today))

The sublists contain MP IDs in chunks of 1000.

In [ ]:
chunk_size = 1000
sublists = [mp_ids[i:i+chunk_size] for i in range(0, len(mp_ids), chunk_size)]

# MPRester.supported_properties
properties = ['energy_per_atom', 'formation_energy_per_atom',
              'e_above_hull', 'icsd_ids',
             'material_id', 'structure']

data = []
# Get all materials from MP by mpid
with MPRester() as m:  # use api_key arg or set up with pmg command line tool
    for sublist in sublists:
        data += m.query({"material_id":{"$in": sublist}}, properties=properties)

In [ ]:
dumpfn(data, "mp_fblock_%s.json" % (today))

### Access a small sample dataset
We want to be responsible users of the MAPI, so to test out pumml models we can work with small MP datasets that are already downloaded.
Download a small example set here: https://figshare.com/articles/500_example_structures_from_Materials_Project/12252962

In [ ]:
data = loadfn('mp_example_dataset_042420.json')  # json file must be in same directory as this notebook

### Data pre-processing

In [ ]:
df_pumml = pd.DataFrame(data)

If a structure matches an experimentally determined crystal structure from the Inorganic Crystal Structure Database (ICSD), then it is labeled as synthesized (1). 

In [ ]:
# PU_label is 1 (0) if experimental crystal structure exists (doesn't exist)
df_pumml['PU_label'] = df_pumml.icsd_ids.apply(lambda x: 0 if not x else 1)

In [ ]:
df_pumml.PU_label.value_counts()

In [ ]:
df_pumml.head()

### Manual featurization
The thermodynamic data from DFT will do most of the heavy lifting for model predictions, but we'll add some additional features so we can look for trends later.

In [ ]:
from matminer.featurizers.structure import DensityFeatures, GlobalSymmetryFeatures
from matminer.featurizers.composition import Meredig, CohesiveEnergy

dfeat = DensityFeatures()
symmfeat = GlobalSymmetryFeatures()
cefeat = CohesiveEnergy()

In [ ]:
densities, vpas, pfs, sgns, cenergies, mpids = [], [], [], [], [], []
for row in df_pumml.itertuples():
    f1, f2, f3, f4, f5 = 0, 0, 0, 0, 0
    try:
        s = row[6]
        mpid = row[5]
        mpids.append(mpid)
        fepa = row[2]
        c = s.composition

        f = dfeat.featurize(s)
        f1 = f[0]
        f2 = f[1]
        f3 = f[2]

        f = symmfeat.featurize(s)
        f4 = f[0]
        
        f = cefeat.featurize(c, formation_energy_per_atom=fepa)
        f5 = f[0]
    except:
        pass
    
    densities.append(f1)
    vpas.append(f2)
    pfs.append(f3)
    sgns.append(f4)
    cenergies.append(f5)

In [ ]:
df_feats = pd.DataFrame({'material_id': mpids, 'density': densities, 'vpa': vpas, 'packing fraction': pfs,
                         'spacegroup_num': sgns, 'cohesive_energy': cenergies})

In [ ]:
df_pumml['structure'] = df_pumml.structure.apply(lambda x: x.as_dict())

In [ ]:
df_featurized = df_pumml.merge(df_feats, on='material_id')

In [ ]:
# Add a column indicating if cohesive energy was computed successfully
df_featurized['no_cohesive_energy'] = df_featurized.cohesive_energy.apply(lambda x: 1 if x > 0 else 0)

In [ ]:
df_featurized = df_featurized[df_featurized['spacegroup_num'] > 0]  # ignore compounds that failed to featurize

In [ ]:
fillval = df_featurized.query('cohesive_energy>0').cohesive_energy.mean()

In [ ]:
# Fill missing cohesive energies with the mean value
df_featurized['cohesive_energy'] = df_featurized.cohesive_energy.apply(lambda x: x if x > 0 else fillval)

Write the processed data to a json file that will be read by pumml.

In [ ]:
df_featurized.to_json('pumml_mp_dataset.json')

### Train a pumml model

In [ ]:
from pumml.learners import PULearner

In [ ]:
pul = PULearner()

# Set hyperparameters to reasonable defaults
n_splits = 5 # kfold CV
n_repeats = 3  # Repeat the entire kfold CV n times for averaging
n_bags = 100  # bags for bootstrap aggregating.

pu_stats = pul.cv_baggingDT('pumml_mp_dataset.json', splits=n_splits, repeats=n_repeats, bags=n_bags)

### Take a look at synthetic accessibility scores of unlabeled compounds

In [ ]:
df1 = pul.df_U.copy()
df1['synth_score'] = pu_stats['prob']

In [ ]:
# Inspect compounds with highest synthetic accessibility scores
df1.sort_values(by='synth_score', ascending=False)[:20]

In [ ]:
df1.synth_score.plot(kind='kde')

In [ ]:
df1.query('synth_score>0.5').spacegroup_num.value_counts()

In [ ]:
# Save synthesis scores
df1.drop(columns=['structure', 'icsd_ids', 'PU_label']).to_json('synth_scores.json')

In [ ]:
print('Number of predicted positives: ', len(df1.query('synth_score > 0.5')))
print('Percentage of predicted positives: ', round(len(df1.query('synth_score > 0.5')) / len(df1), 3))

In [ ]:
pul.get_feat_importances(plot_format='png')